**hf login**

In [1]:
from huggingface_hub import login
hf_token= "past your hf token here"
login(token = hf_token)

**install required package**

In [2]:
!pip install -q -U bitsandbytes torch transformers accelerate einops wandb
!pip install -q -U trl
!pip install -q -U git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 817.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

**Declare imports**

In [3]:
import os
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

**set model_id and BitsAndBytes config**

In [5]:
model_id = "meta-llama/Llama-3.2-1B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

**Set LoRA config**

In [6]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "v_proj"],
    task_type = "CAUSAL_LM"
)

**Tokenizer & load model**

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map = {'':0}
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

**load required dataset**

In [16]:
from datasets import load_dataset
data = load_dataset("HiuXB/Welding-Handbook-QA")
dataset = data.map(lambda samples: tokenizer(['quetion','answer']))

Map:   0%|          | 0/16389 [00:00<?, ? examples/s]

ArrowInvalid: Column 3 named input_ids expected length 1000 but got length 2

**prepare data for training**

In [36]:
def formatting_func(example):
  text = f"Quetion: {example['question']}\nAnswer :{example['answer']}{tokenizer.eos_token}"
  return text

In [37]:
data['train']

Dataset({
    features: ['question', 'answer', 'id'],
    num_rows: 16389
})

**set training arg**

In [38]:
tainer = SFTTrainer(
    model=model,
    train_dataset = data['train'],
    args = transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        logging_steps=1,
        warmup_steps=2,
        max_steps=100,
        fp16=True,
        optim='paged_adamw_8bit',
        output_dir='output',
        report_to='none'
    ),
    peft_config=lora_config,
    formatting_func=formatting_func
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Applying formatting function to train dataset:   0%|          | 0/16389 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/16389 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/16389 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/16389 [00:00<?, ? examples/s]

**initialize training train**

In [40]:
tainer.train()

Step,Training Loss
1,2.228700
2,2.122700
3,2.171100
4,2.159300
5,2.542000
6,2.134600
7,2.091000
8,2.262200
9,2.223500
10,2.429400


TrainOutput(global_step=100, training_loss=2.0310260784626006, metrics={'train_runtime': 170.5645, 'train_samples_per_second': 2.345, 'train_steps_per_second': 0.586, 'total_flos': 721557257625600.0, 'train_loss': 2.0310260784626006, 'epoch': 0.024406614192446153})

**Merge trained model into base model**

In [45]:
final_model=PeftModel.from_pretrained(model,
                                      "./output/checkpoint-100")
# Correct method to merge LoRA weights into the base model
merge_model = final_model.merge_and_unload()

save_path = "./output/FinalModel"
merge_model.save_pretrained(save_path)
AutoTokenizer.from_pretrained(model_id).save_pretrained(save_path)

print(f"✅ Merged model saved at: {save_path}")

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


✅ Merged model saved at: ./output/FinalModel


**Test**

In [51]:
tok = AutoTokenizer.from_pretrained("./output/FinalModel")
mdl = AutoModelForCausalLM.from_pretrained(
    "./output/FinalModel",
    device_map={'':0}
)

prompt = """Explain the transient liquid-phase (TLP) bonding method used for joining silicon-base ceramics and describe why it is more challenging to adapt TLP bonding when joining ceramics with a metallic interlayer. Include in your answer the role of the tri-layer interlayer design and the key factors that influence the isothermal solidification of the liquid phase in these joints.\nAnswer:"""
inputs = tok(prompt, return_tensors="pt").to(mdl.device)

out = mdl.generate(**inputs, max_new_tokens=128)
print(tok.decode(out[0], skip_special_tokens=True))


The tokenizer you are loading from './output/FinalModel' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Explain the transient liquid-phase (TLP) bonding method used for joining silicon-base ceramics and describe why it is more challenging to adapt TLP bonding when joining ceramics with a metallic interlayer. Include in your answer the role of the tri-layer interlayer design and the key factors that influence the isothermal solidification of the liquid phase in these joints.
Answer: Transient liquid-phase (TLP) bonding method used for joining silicon-base ceramics and describe why it is more challenging to adapt TLP bonding when joining ceramics with a metallic interlayer. Include in your answer the role of the tri-layer interlayer design and the key factors that influence the isothermal solidification of the liquid phase in these joints.
Transitive liquid-phase (TLP) bonding method used for joining silicon-base ceramics and describe why it is more challenging to adapt TLP bonding when joining ceramics with a metallic interlayer. Include in your answer the role of the tri-layer interlayer